In [1]:
import hail as hl
hl.init(log='/tmp/foo.log')

Running on Apache Spark version 2.4.3
SparkUI available at http://ukbb-nb-1-m.c.ukbb-round2.internal:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.19-c6ec8b76eb26
LOGGING: writing to /tmp/foo.log


In [2]:
wd = 'gs://nbaya/risk_gradients/'

variant_set = 'qc_pos'
maf = 0.05
# whether to subset to the intersection of SNPs in the EUR 1KG SNPs suggested by PRScs
use_1kg_eur_hm3_snps = True
h2 = 0.75
pi = 0.001
chrom='all'

In [37]:
mt = hl.read_matrix_table(
    wd+f'genotypes{("" if chrom is "all" else f".chr{chrom}")}.all.{variant_set}.maf_{maf}{".1kg_eur_hm3" if use_1kg_eur_hm3_snps else ""}.mt')

n_train = int(300e3)
seed = 1
train = hl.import_table(
    wd+f'iid.sim.train.n_{n_train}.seed_{seed}.tsv.bgz').key_by('s')

sim_cols = hl.read_table(
    wd+f'sim.cols{("" if chrom is "all" else f".chr{chrom}")}.all.{variant_set}.maf_{maf}.h2_{h2}.pi_{pi}{".1kg_eur_hm3" if use_1kg_eur_hm3_snps else ""}.ht')
sim = mt.annotate_cols(sim_y=sim_cols[mt.s].y)                       

2019-08-27 19:39:00 Hail: INFO: Reading table with no type imputation
  Loading column 's' as type 'str' (type not specified)
  Loading column 'label_100000' as type 'str' (type not specified)
  Loading column 'label_50000' as type 'str' (type not specified)
  Loading column 'label_20000' as type 'str' (type not specified)
  Loading column 'label_10000' as type 'str' (type not specified)
  Loading column 'label_5000' as type 'str' (type not specified)



In [39]:
n_train_sub = int(5e3)
subset=0

In [40]:
train_sub = train.filter(
    train[f'label_{n_train_sub}'] == str(subset))
sim_sub = sim.filter_cols(hl.is_defined(train_sub[sim.s]))

In [41]:
sim_sub.describe()

----------------------------------------
Global fields:
    None
----------------------------------------
Column fields:
    's': str
    'isFemale': bool
    'age': int32
    'age_squared': int32
    'age_isFemale': int32
    'age_squared_isFemale': int32
    'PC1': float64
    'PC2': float64
    'PC3': float64
    'PC4': float64
    'PC5': float64
    'PC6': float64
    'PC7': float64
    'PC8': float64
    'PC9': float64
    'PC10': float64
    'PC11': float64
    'PC12': float64
    'PC13': float64
    'PC14': float64
    'PC15': float64
    'PC16': float64
    'PC17': float64
    'PC18': float64
    'PC19': float64
    'PC20': float64
    'sim_y': float64
----------------------------------------
Row fields:
    'locus': locus<GRCh37>
    'alleles': array<str>
    'rsid': str
    'varid': str
    'A1_1kg': str
    'A2_1kg': str
    'AF': float64
----------------------------------------
Entry fields:
    'dosage': float64
----------------------------------------
Column key: ['s']
Ro

In [45]:
sim_sub.count()

(1008856, 5000)

In [13]:
variants = hl.read_table('gs://nbaya/split/qc_pos_variants.ht')
# variants = variants.annotate(**hl.parse_variant(variants.v))
# variants = variants.key_by('locus','alleles') 
gt0 = hl.import_bgen(path='gs://fc-7d5088b4-7673-45b5-95c2-17ae00a04183/imputed/ukb_imp_chr'+str(set(range(1,23))).replace(' ','')+'_v3.bgen',
                     entry_fields=['GT'],
                     n_partitions = 1000,
                     sample_file = 'gs://ukb31063/ukb31063.autosomes.sample',
                     variants=variants)

2019-08-27 18:59:30 Hail: INFO: Number of BGEN files parsed: 22
2019-08-27 18:59:30 Hail: INFO: Number of samples in BGEN files: 487409
2019-08-27 18:59:30 Hail: INFO: Number of variants across all BGEN files: 93095623
2019-08-27 19:01:58 Hail: INFO: Number of BGEN files parsed: 22
2019-08-27 19:01:58 Hail: INFO: Number of samples in BGEN files: 487409
2019-08-27 19:01:58 Hail: INFO: Number of variants across all BGEN files: 93095623


In [15]:
gt0.describe()

----------------------------------------
Global fields:
    None
----------------------------------------
Column fields:
    's': str
----------------------------------------
Row fields:
    'locus': locus<GRCh37>
    'alleles': array<str>
    'rsid': str
    'varid': str
----------------------------------------
Entry fields:
    'GT': call
----------------------------------------
Column key: ['s']
Row key: ['locus', 'alleles']
----------------------------------------


In [42]:
mt_joined = gt0[sim_sub.row_key, sim_sub.col_key]
sim_sub1 = sim_sub.annotate_entries(GT=mt_joined.GT)

In [43]:
sim_sub1.describe()

----------------------------------------
Global fields:
    None
----------------------------------------
Column fields:
    's': str
    'isFemale': bool
    'age': int32
    'age_squared': int32
    'age_isFemale': int32
    'age_squared_isFemale': int32
    'PC1': float64
    'PC2': float64
    'PC3': float64
    'PC4': float64
    'PC5': float64
    'PC6': float64
    'PC7': float64
    'PC8': float64
    'PC9': float64
    'PC10': float64
    'PC11': float64
    'PC12': float64
    'PC13': float64
    'PC14': float64
    'PC15': float64
    'PC16': float64
    'PC17': float64
    'PC18': float64
    'PC19': float64
    'PC20': float64
    'sim_y': float64
----------------------------------------
Row fields:
    'locus': locus<GRCh37>
    'alleles': array<str>
    'rsid': str
    'varid': str
    'A1_1kg': str
    'A2_1kg': str
    'AF': float64
----------------------------------------
Entry fields:
    'dosage': float64
    'GT': call
----------------------------------------
Colum

In [65]:
gt0.GT.describe()

--------------------------------------------------------
Type:
        call
--------------------------------------------------------
Source:
Index:
    ['row', 'column']
--------------------------------------------------------


In [48]:
hl.export_plink(dataset=sim_sub1,
               output=wd+f'sim.5k{("" if chrom is "all" else f".chr{chrom}")}.all.{variant_set}.maf_{maf}{".1kg_eur_hm3" if use_1kg_eur_hm3_snps else ""}',
               call=sim_sub1.GT,
               fam_id='0',
               ind_id=sim_sub1.s,
               pat_id='0',
               mat_id='0',
               is_female=sim_sub1.isFemale,
               pheno=sim_sub1.sim_y,
               varid=sim_sub1.rsid,
               cm_position=0.0)

2019-08-27 20:43:27 Hail: INFO: merging 5001 files totalling 1.2G...
2019-08-27 20:44:18 Hail: INFO: while writing:
    gs://nbaya/risk_gradients/sim.5k..all.qc_pos.maf_0.05.1kg_eur_hm3.bed
  merge time: 50.720s
2019-08-27 20:44:20 Hail: INFO: merging 5000 files totalling 28.7M...
2019-08-27 20:44:36 Hail: INFO: while writing:
    gs://nbaya/risk_gradients/sim.5k..all.qc_pos.maf_0.05.1kg_eur_hm3.bim
  merge time: 16.906s
2019-08-27 20:44:38 Hail: INFO: merging 4 files totalling 139.2K...
2019-08-27 20:44:38 Hail: INFO: while writing:
    gs://nbaya/risk_gradients/sim.5k..all.qc_pos.maf_0.05.1kg_eur_hm3.fam
  merge time: 404.393ms
2019-08-27 20:44:38 Hail: INFO: wrote 1008856 variants and 5000 samples to 'gs://nbaya/risk_gradients/sim.5k..all.qc_pos.maf_0.05.1kg_eur_hm3'


In [53]:
sim_sub1 = hl.variant_qc(sim_sub1)

In [59]:
sim_sub1 = sim_sub1.rename({'AF':'alt_AF'})

In [ ]:
sim_sub1.rows().export(wd+'sim.rows.')

In [61]:
sim_sub1 = sim_sub1.rename({'A1_maf':'A1_AF'})

In [62]:
sim_sub1.rows().export(wd+'sim.rows.5k.all.qc_pos.maf_0.05.1kg_eur_hm3')

2019-08-27 21:04:15 Hail: INFO: Coerced sorted dataset


KeyboardInterrupt: 

In [68]:
info = hl.import_table('gs://nbaya/risk_gradients/ukb.maf_info.tsv.bgz')

2019-08-27 21:19:29 Hail: INFO: Reading table with no type imputation
  Loading column 'locus' as type 'str' (type not specified)
  Loading column 'alleles' as type 'str' (type not specified)
  Loading column 'varid' as type 'str' (type not specified)
  Loading column 'rsid' as type 'str' (type not specified)
  Loading column 'minor_allele' as type 'str' (type not specified)
  Loading column 'maf' as type 'str' (type not specified)
  Loading column 'info' as type 'str' (type not specified)



In [71]:
info = info.annotate(locus=hl.parse_locus(s=info.locus,reference_genome='GRCh37'))

In [82]:
sim_sub_rows = sim_sub1.rows().key_by('locus','rsid')
info = info.key_by('locus','rsid')
info = info.filter(hl.is_defined(sim_sub_rows[info.locus,info.rsid]))

In [83]:
info.count()

2019-08-27 21:54:28 Hail: INFO: Finished type imputation
  Loading column 'locus' as type 'str' (imputed)
  Loading column 'alleles' as type 'str' (imputed)
  Loading column 'varid' as type 'str' (imputed)
  Loading column 'rsid' as type 'str' (imputed)
  Loading column 'minor_allele' as type 'str' (imputed)
  Loading column 'maf' as type 'float64' (imputed)
  Loading column 'info' as type 'float64' (imputed)
2019-08-27 21:54:37 Hail: INFO: Finished type imputation
  Loading column 'locus' as type 'str' (imputed)
  Loading column 'alleles' as type 'str' (imputed)
  Loading column 'varid' as type 'str' (imputed)
  Loading column 'rsid' as type 'str' (imputed)
  Loading column 'minor_allele' as type 'str' (imputed)
  Loading column 'maf' as type 'float64' (imputed)
  Loading column 'info' as type 'float64' (imputed)
2019-08-27 21:54:52 Hail: INFO: Ordering unsorted dataset with network shuffle
2019-08-27 21:55:07 Hail: INFO: Coerced sorted dataset


1010906

In [87]:
sim_sub.count()

(1008856, 5000)

In [88]:
info = info.annotate(A1 = sim_sub_rows[info.locus,info.rsid].alleles[0],
                     A2 = sim_sub_rows[info.locus,info.rsid].alleles[1])

In [89]:
info.export(wd+'sim.variant_info.all.qc_pos.maf_0.05.1kg_eur_hm3.tsv.gz')

2019-08-27 22:07:00 Hail: INFO: Ordering unsorted dataset with network shuffle
2019-08-27 22:07:17 Hail: INFO: Coerced sorted dataset
2019-08-27 22:07:21 Hail: INFO: Coerced sorted dataset
2019-08-27 22:07:25 Hail: INFO: Coerced sorted dataset
2019-08-27 22:07:29 Hail: INFO: Coerced sorted dataset
2019-08-27 22:07:34 Hail: INFO: Coerced sorted dataset
2019-08-27 22:15:52 Hail: INFO: merging 17 files totalling 19.7M...
2019-08-27 22:15:53 Hail: INFO: while writing:
    gs://nbaya/risk_gradients/sim.variant_info.all.qc_pos.maf_0.05.1kg_eur_hm3.tsv.gz
  merge time: 953.821ms
